In [1]:
import os

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.part_number/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：10


['30250S',
 '30250SX',
 '30420S',
 '30430S',
 '30460S',
 '30460SA',
 '30460SB',
 '30470S',
 '30470SX',
 '30490S']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'dopartsearch': '1',
        'partsearch[partnum][partsearch_007]': input_.loc[a, 'Part Number'],
        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'Manufacturer'],
        'partsearch[parttype][partsearch_007]': input_.loc[a, 'Type Code'],
        'partsearch[do][partsearch_007]': '1',
        'func': 'sendparttabsearch',
        'giveback': json.dumps({}),
        'api_json_request': '1'}

b = 0
while True:
    b += 1
    print(b)
    try:
        check_url = input_.loc[a, 'Url']
        
        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(10, 10))
        
        if resp.status_code == 200:
            dict_ = resp.json()
            
            if 'redirect_to_url' not in dict_ and not dict_['giveback'] and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

dict_

1


{'giveback': [],
 'searchnoderesults': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;partsearch&quot;,&quot;idepth&quot;:0,&quot;parttype&quot;:&quot;5756&quot;,&quot;catalogname&quot;:&quot;MELLING&quot;,&quot;partnumber&quot;:&quot;30250S&quot;,&quot;nodetype&quot;:&quot;parttabsearch&quot;,&quot;expanded&quot;:true,&quot;loaded&quot;:true}" /><input type="hidden" autocomplete="off" name="navnodeunique[partsearch-5756]" id="navnodeunique[partsearch-5756]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nexpandedicon" id="navicon[__GIP__1__]"><a href="/en/parts/melling,30250S,timing+chain+&amp;+component+kit,5756" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1__&quot;); return false;" tabindex="-1"><img src="/Images/1px

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"partsearch","idepth":0,"parttype":"5756","catalogname":"MELLING","partnumber":"30250S","nodetype":"parttabsearch","expanded":true,"loaded":true}'/>
   <input autocomplete="off" id="navnodeunique[partsearch-5756]" name="navnodeunique[partsearch-5756]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nexpandedicon" id="navicon[__GIP__1__]">
        <a href="/en/parts/melling,30250S,timing+chain+&amp;+component+kit,5756" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
       <td class="nlabel">
        <a class="navlabellink nvoffset nnor

In [9]:
list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

list_tbody

[<Element tbody at 0x1294f2140>]

In [10]:
import re

list_list_note = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]') for tbody in list_tbody]
list_list_note = [[etree.tostring(note).decode('utf-8') for note in list_note] if list_note else [] for list_note in list_list_note]
list_list_note = [[re.sub(r'<.*?>', '', note) for note in list_note] for list_note in list_list_note]
list_list_note = [[re.sub(r' +', ' ', note).strip() for note in list_note] for list_note in list_list_note]

list_list_note

[[]]

In [11]:
list_dict_note = []
for list_note in list_list_note:
    dict_note = {}
    for i, note in enumerate(list_note):
        dict_note[str(i)] = note

    list_dict_note.append(dict_note)

list_dict_note

[{}]

In [12]:
list_list_note = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]') for tbody in list_tbody]
list_list_note = [[etree.tostring(text).decode('utf-8') for note in list_note for text in note.xpath('./span[@class="listing-footnote-text"]')] if list_note else [] for list_note in list_list_note]
list_list_note = [[re.sub(r'<.*?>', '', note) for note in list_note] for list_note in list_list_note]
list_list_note = [[re.sub(r' +', ' ', note).strip() for note in list_note] for list_note in list_list_note]

list_list_note

[['Category: Timing Chain &amp; Component Kit']]

In [13]:
list_dict_note_2 = []
for list_note in list_list_note:
    dict_note = {}
    for i, note in enumerate(list_note):
        dict_note[str(i)] = note

    list_dict_note_2.append(dict_note)

list_dict_note_2

[{'0': 'Category: Timing Chain &amp; Component Kit'}]

In [14]:
list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict

[{'groupindex': '__GIP__3__',
  'carcode': 0,
  'parttype': '5756',
  'partkey': '8304960',
  'opts': {'0-0-0-1': {'warehouse': '72901',
    'whpartnum': 'MEL 3-0250S@NK',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}}]

In [15]:
list_type_code = [dict_['parttype'] for dict_ in list_dict]

list_type_code

['5756']

In [16]:
import numpy as np

array_index_type_code = np.where(np.array(list_type_code) == input_.loc[a, 'Type Code'])

array_index_type_code

(array([0]),)

In [17]:
list_part_code = [dict_['partkey'] for dict_ in list_dict]

list_part_code

['8304960']

In [18]:
list_list_info = [tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]') for tbody in list_tbody]
list_info = [list_info[0].xpath('./@href')[0].strip() if list_info else '' for list_info in list_list_info]
list_info = [info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={part_code}&cc=0&pt={type_code}&Lennon=1''' for info, part_code, type_code in zip(list_info, list_part_code, list_type_code)]

list_info

['https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756']

In [19]:
list_dict = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict

[{'partnumber': '30250S',
  'catalogname': 'MELLING',
  'belongstolisting': '__GIP__2__',
  'sortgroup': 0,
  'sortgrouptext': '',
  'paramdesc': '',
  'iskitparent': True}]

In [20]:
array_index_part_number = np.where(np.array([dict_['partnumber'] for dict_ in list_dict]) == input_.loc[a, 'Part Number'])

array_index_part_number

(array([0]),)

In [21]:
array_index_manufacturer = np.where(np.array([dict_['catalogname'] for dict_ in list_dict]) == input_.loc[a, 'Manufacturer'])

array_index_manufacturer

(array([0]),)

In [22]:
array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

array_index_part

array([0])

In [23]:
if len(array_index_part) == 1:
    index_part = array_index_part[0]

index_part

0

In [24]:
choose = list_dict[index_part]['paramdesc'].strip()

choose

''

In [25]:
list_list_text = [td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')]

list_list_text

[[]]

In [26]:
list_oem = []
for list_text in list_list_text:
    if list_text:
        for text in list_text:
            list_oem += text.split(',')
        
list_oem = [oem.strip() for oem in list_oem]
    
while '' in list_oem:
    list_oem.remove('')

oem = ';'.join(sorted(list(set(list_oem))))

oem

''

In [27]:
list_src = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
dict_ = json.loads(list_src[0].xpath('./@value')[0]) if list_src else {}

dict_

{'Size': 'Thumb',
 'SlotIndex': 0,
 'Slots': [{'TitlePhrase': None,
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/583/3-0250S-1-KIT__ra_m.jpg',
    'Popup': '/info/583/3-0250S-1-KIT.jpg',
    'Full': '/info/583/3-0250S-1-KIT.jpg'}},
  {'TitlePhrase': '<b>[Timing Chain Guide / Damper]</b>',
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/583/BT7015-2-SID__ra_m.jpg',
    'Popup': '/info/583/BT7015-2-SID.jpg',
    'Full': '/info/583/BT7015-2-SID.jpg'}},
  {'TitlePhrase': '<b>[Timing Chain Guide / Damper]</b>',
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/583/BG5515-1-SID__ra_m.jpg',
    'Popup': '/info/583/BG5515-1-SID.jpg',
    'Full': '/info/583/BG5515-1-SID.jpg'}},
  {'TitlePhrase': '<b>[Timing Chain]</b>',
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/583/715F-1-ANG__ra_m.jpg',
    'Popup': '/info/583/715F-1-ANG.jpg',
    'Full': '/info/583/715F-1-ANG.jpg'

In [28]:
dict_src = {}
if dict_:
    list_src = dict_['Slots']
    for i, src in enumerate(list_src):
        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''

dict_src

{'0': 'https://www.rockauto.com/info/583/3-0250S-1-KIT.jpg',
 '1': 'https://www.rockauto.com/info/583/BT7015-2-SID.jpg',
 '2': 'https://www.rockauto.com/info/583/BG5515-1-SID.jpg',
 '3': 'https://www.rockauto.com/info/583/715F-1-ANG.jpg',
 '4': 'https://www.rockauto.com/info/583/S906A-1-ANG.jpg',
 '5': 'https://www.rockauto.com/info/583/BT5514-1-ANG.jpg',
 '6': 'https://www.rockauto.com/info/583/717F-1-ANG.jpg',
 '7': 'https://www.rockauto.com/info/583/BT7016-1-TOP.jpg',
 '8': 'https://www.rockauto.com/info/583/S899-1-ANG.jpg',
 '9': 'https://www.rockauto.com/info/583/S905-1-FRO.jpg',
 '10': 'https://www.rockauto.com/info/583/BG7019-2-SID.jpg'}

In [29]:
price = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
price = etree.tostring(price[0]).decode('utf-8') if price else ''
price = re.sub(r'<.*?>', '', price)
price = re.sub(r' +', ' ', price).strip()
dict_price = {'0': price}

dict_price

{'0': '$171.79'}

In [30]:
td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')[0]
list_option = [etree.tostring(option).decode('utf-8') for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
list_option = [re.sub(r'<.*?>', '', option) for option in list_option]
list_option = [re.sub(r' +', ' ', option).strip() for option in list_option]
if list_option:
    dict_price['1'] = {}
    for i, option in enumerate(list_option):
        dict_price['1'][str(i)] = option

dict_price

{'0': '$171.79'}

In [31]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': input_.loc[a, 'Type'],
                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Vehicle': '',
                         'OEM': oem,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'Info': list_info[index_part],
                         'Choose': choose,
                         'Json_Price': json.dumps(dict_price),
                         'Json_Note 1': json.dumps(list_dict_note[index_part]),
                         'Json_Note 2': json.dumps(list_dict_note_2[index_part]),
                         'Group': input_.loc[a, 'Group'],
                         'Type Code': input_.loc[a, 'Type Code'],
                         'Part Code': list_part_code[index_part],
                         'JOIN_MPNTCPC': f'''{input_.loc[a, 'Manufacturer']};{input_.loc[a, 'Part Number']};{input_.loc[a, 'Type Code']};{list_part_code[index_part]}'''}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Manufacturer,Part Number,Vehicle,OEM,Picture,Url,Json_Src,Info,Choose,Json_Price,Json_Note 1,Json_Note 2,Group,Type Code,Part Code,JOIN_MPNTCPC
0,1,Timing Chain & Component Kit,MELLING,30250S,,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,,"{""0"": ""$171.79""}",{},"{""0"": ""Category: Timing Chain &amp; Component ...",Engine,5756,8304960,MELLING;30250S;5756;8304960


In [32]:
crawler_status = 'ok'

crawler_status

'ok'

In [33]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [34]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Type,Manufacturer,Part Number,Vehicle,OEM,Picture,Url,Json_Src,Info,Choose,Json_Price,Json_Note 1,Json_Note 2,Group,Type Code,Part Code,JOIN_MPNTCPC
0,1,Timing Chain & Component Kit,MELLING,30250S,,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,,"{""0"": ""$171.79""}",{},"{""0"": ""Category: Timing Chain &amp; Component ...",Engine,5756,8304960,MELLING;30250S;5756;8304960


In [35]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
